In [ ]:
'''
importing libraries and initialising the variables and values
'''
# importing libraries
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import glob  # to find files
import seaborn as sns
# Seaborn library for bar chart
#import seaborn as sns

# Libraries for TensorFlow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, layers

# Library for Transfer Learning
from tensorflow.keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

print("Importing libraries completed.")



train_folder ="Data/"
# variables for image size
img_width = 100
img_height = 100

# variable for model
batch_size = 32
epochs = 10

print("Variable declaration completed.")

# listing the folders containing images

# Train Dataset
train_class_names = os.listdir(train_folder)
print("Train class names: %s" % (train_class_names))
# print("\n")



print("\nDataset class name listing completed.")

# declaration of functions


# Declaring variables
x = []  # to store array value of the images
y = []  # to store the labels of the images

for folder in os.listdir(train_folder):
    image_list = os.listdir(train_folder + "/" + folder)
    for img_name in image_list:
        # Loading images
        img = image.load_img(train_folder + "/" + folder + "/" + img_name, target_size=(img_width, img_height))

        # Converting to arrary
        img = image.img_to_array(img)

        # Transfer Learning: this is to apply preprocess of VGG16 model to our images before passing it to VGG16
        img = preprocess_input(img)  # Optional step

        # Appending the arrarys
        x.append(img)  # appending image array
        y.append(train_class_names.index(folder))  # appending class index to the array

print("Preparing Training Dataset Completed.")

# Training Dataset
print("Training Dataset")

x = np.array(x)  # Converting to np arrary to pass to the model
print(x.shape)

y = to_categorical(y)  # onehot encoding of the labels
# print(y)
print(y.shape)


In [ ]:
'''
this moudule imports and train the model using VGG16 algorithm

'''
print("Summary of default VGG16 model.\n")
# we are using VGG16 for transfer learnin here. So we have imported it
from tensorflow.keras.applications import VGG16

model_vgg16 = VGG16(weights='imagenet')
model_vgg16.summary()

print("Summary of Custom VGG16 model.\n")
input_layer = layers.Input(shape=(img_width, img_height, 3))
model_vgg16 = VGG16(weights='imagenet', input_tensor=input_layer, include_top=False)
model_vgg16.summary()
last_layer = model_vgg16.output
flatten = layers.Flatten()(last_layer)
output_layer = layers.Dense(6, activation='softmax')(flatten)
model = models.Model(inputs=input_layer, outputs=output_layer)

model.summary()
for layer in model.layers[:-1]:
    layer.trainable = False
model.summary()

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=0)
print("Splitting data for train and test completed.")

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Model compilation completed.")

history2 = model.fit(xtrain, ytrain, epochs=epochs, batch_size=batch_size, verbose=True, validation_data=(xtrain, ytrain))

print("Fitting the model completed.")

model.save("Vggmodel.h5")

acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']
epochs = range(len(acc))

plt.plot(epochs, acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot Model Loss
loss_train = history2.history['loss']
loss_val = history2.history['val_loss']
plt.plot(epochs, loss_train, label='Training Loss')
plt.plot(epochs, loss_val, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

y_pred = model.predict(xtrain)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)
y_test=np.argmax(ytrain, axis=1)
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print(cm)

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix of Vgg16 ')
plt.show()


In [ ]:
'''

this will predict the result of the image uploaded by the user whether it is a degragable or non-degragable item by precdition result

'''
from flask import Flask, render_template, flash, request, session
import warnings
import os
import mysql.connector

app = Flask(__name__)
app.config.from_object(__name__)
app.config['SECRET_KEY'] = '<add secretkey>'


@app.route("/")
def homepage():
    return render_template('index.html')


@app.route("/Prediction")
def Prediction():
    return render_template('Prediction.html')


@app.route("/predict", methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        import tensorflow as tf
        import numpy as np
        import cv2
        from keras.preprocessing import image
        file = request.files['file']
        file.save('static/upload/Test.jpg')
        org = 'static/upload/Test.jpg'

        img1 = cv2.imread('static/upload/Test.jpg')

        dst = cv2.fastNlMeansDenoisingColored(img1, None, 10, 10, 7, 21)
        noi = 'static/upload/noi.jpg'
        cv2.imwrite(noi, dst)

        classifierLoad = tf.keras.models.load_model('Vggmodel.h5')
        test_image = image.load_img('static/upload/Test.jpg', target_size=(100, 100))
        test_image = np.expand_dims(test_image, axis=0)
        result = classifierLoad.predict(test_image)
        print(result)

        result = classifierLoad.predict(test_image)
        print(result)
        result = np.argmax(result, axis=1)

        print(result)

        out = ''
        pre = ''
        if result[0] == 0:
            print("cardboard")
            out = "cardboard"
            pre = "Degradable"
        elif result[0] == 1:
            print("glass")
            out = "glass"
            pre = "Non-Degradable"
        elif result[0] == 2:
            print("metal")
            out = "metal"
            pre = "Non-Degradable"
        elif result[0] == 3:
            print("paper")
            out = "paper"
            pre = "Degradable"

        elif result[0] == 4:
            print("plastic")
            out = "plastic"
            pre = "Non-Degradable"
        elif result[0] == 5:
            print("trash")
            out = "trash"
            pre = "Degradable"

        else:
            print("Invalid")

        return render_template('Result.html', res=out, pre=pre, org=org, noi=noi)



In [ ]:
'''

this module will connect the camera in the device for the user input and give the prediction result to the user

'''
@app.route("/Camera")
def Camera():
    import warnings
    import cv2
    import numpy as np
    import os
    import time
    args = {"confidence": 0.5, "threshold": 0.3}
    flag = False

    labelsPath = "./yolo-coco/coco.names"
    LABELS = open(labelsPath).read().strip().split("\n")
    final_classes = ['bottle', 'wine glass', 'cup', 'cell phone', 'book']

    np.random.seed(42)
    COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
                               dtype="uint8")

    weightsPath = os.path.abspath("./yolo-coco/yolov3-tiny.weights")
    configPath = os.path.abspath("./yolo-coco/yolov3-tiny.cfg")

    # print(configPath, "\n", weightsPath)

    net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    vs = cv2.VideoCapture(0)
    writer = None
    (W, H) = (None, None)

    flag = True

    flagg = 0

    while True:
        # read the next frame from the file
        (grabbed, frame) = vs.read()

        # if the frame was not grabbed, then we have reached the end
        # of the stream
        if not grabbed:
            break

        # if the frame dimensions are empty, grab them
        if W is None or H is None:
            (H, W) = frame.shape[:2]

        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
                                     swapRB=True, crop=False)
        net.setInput(blob)
        start = time.time()
        layerOutputs = net.forward(ln)
        end = time.time()

        # initialize our lists of detected bounding boxes, confidences,
        # and class IDs, respectively
        boxes = []
        confidences = []
        classIDs = []

        # loop over each of the layer outputs
        for output in layerOutputs:
            # loop over each of the detections
            for detection in output:
                # extract the class ID and confidence (i.e., probability)
                # of the current object detection
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]

                # filter out weak predictions by ensuring the detected
                # probability is greater than the minimum probability
                if confidence > args["confidence"]:
                    # scale the bounding box coordinates back relative to
                    # the size of the image, keeping in mind that YOLO
                    # actually returns the center (x, y)-coordinates of
                    # the bounding box followed by the boxes' width and
                    # height
                    box = detection[0:4] * np.array([W, H, W, H])
                    (centerX, centerY, width, height) = box.astype("int")

                    # use the center (x, y)-coordinates to derive the top
                    # and and left corner of the bounding box
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))

                    # update our list of bounding box coordinates,
                    # confidences, and class IDs
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)
        # apply non-maxima suppression to suppress weak, overlapping
        # bounding boxes
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, args["confidence"],
                                args["threshold"])

        # ensure at least one detection exists
        if len(idxs) > 0:
            # loop over the indexes we are keeping
            for i in idxs.flatten():
                # extract the bounding box coordinates
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])

                if (LABELS[classIDs[i]] in final_classes):

                    flagg += 1
                    # print(flag)
                    # sendmsg("9486365535", " Animal Detected " + LABELS[classIDs[i]])

                    color = [int(c) for c in COLORS[classIDs[i]]]
                    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                    text = "{}: {:.4f}".format(LABELS[classIDs[i]],
                                               confidences[i])
                    cv2.putText(frame, text, (x, y - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    if (flagg == 40):
                        flagg = 0

                        out = LABELS[classIDs[i]]
                        pre = ''
                        if out=="bottle":
                            pre = "Non-Degradable"
                        elif out=="wine glass":
                            pre = "Non-Degradable"
                        elif out=="cup":
                            pre = "Degradable"
                        elif out == "cell phone":
                            pre = "Non-Degradable"
                        elif out == "book":
                            pre = "Degradable"
                        sendmsg("9486365535",'Prediction Result : ' + out + ' ' + pre)
        else:
            flag = True
        cv2.imshow("Output", frame)
        if cv2.waitKey(1) == ord('q'):
            break
    # release the webcam and destroy all active windows
    vs.release()
    cv2.destroyAllWindows()

    return render_template('index.html')


if __name__ == '__main__':
    # app.run(host='0.0.0.0',debug = True, port = 5000)
    app.run(debug=True, use_reloader=True)


